# CEL -- Common Expression Language

## Agenda

- Why CEL?

- About CEL

- Processing

- Lark Implementation Details

- Underlying Python Techniques

- Extension Functions

## Why CEL?

https://github.com/cloud-custodian/cloud-custodian/issues/5759

The Cloud Custodian (C7N) Domain Specific Language (DSL) for policy filters isn't terribly flexible and can be opaque.

The CEL language can be more extensible and easier to reason about without sacrificing performance.

## About CEL

These are the objectives of the CEL implementation.

- Keep it small & fast.

- Make it extensible.

- Developer-friendly.  Similar to C/C++/Java/JavaScript.

CEL is similar to

- Shell `expr` command

- The `jq` command

- Shell `test` command

It has the advantage of being embeddable into other applications, i.e., C7N.

## Processing

CEL works like this, given an expression. ``355. / 113.``

1. Create an environment. The default is generally all we need. 

2. Parse to create an Abstract Syntax Tree. Created by a `lark` parser.  ``Tree('expr', [Tree('literal', [Token]), Tree('literal', [Token])])``.

3. Compile to produce an executable form. This is when external functions can be bound to the expression. (In the pure Python implementation, we do nothing.)

4. (Optional) Create an "activation" with global variables that need to be available. (None for this example.)

4. Apply the executable expression to variables in the activation to compute a response.

In [5]:
import celpy
env = celpy.Environment()
ast = env.compile("355. / 113.")
prgm = env.program(ast)
prgm.evaluate({})

DoubleType(3.1415929203539825)

## Lark Implementation

Build around the Lark parser generator. https://lark-parser.readthedocs.io/en/latest/

Build on top of ``celtypes`` module providing Go-like semantics in Python

-   Lexical Scanning finds language tokens: int, float, identifier, operator, etc.
  
-   Parsing recognizes higher-level (possibly recursive) constructs.

-   Final "evaluator" is a Lark ``Interpreter`` subclass

    -  Walks the tree of parsed constructs

### Lark EBNF Tokens

Look like Regular Expressions. With a few added features.

```
UINT_LIT       : INT_LIT /[uU]/

INT_LIT        : /-?/ /0x/ HEXDIGIT+ | /-?/ DIGIT+

DIGIT          : /[0-9]/

HEXDIGIT       : /[0-9abcdefABCDEF]/
```

### Lark EBNF Grammar Productions

Extended Backus-Naur Form -- summarizes the grammar

```
expr           : conditionalor ["?" conditionalor ":" expr]

conditionalor  : [conditionalor "||"] conditionaland

conditionaland : [conditionaland "&&"] relation

relation       : [relation_lt | relation_le | relation_ge | relation_gt | relation_eq | relation_ne | relation_in] addition
```

### Final Evaluator -- Lark Interpreter Methods

1. Lookup the operator in the CEL operator-to-function mapping. (e.g., addition is "_+_").

2. Visit the children to get the operands.

3. Apply the function to the operands.  Build a ``CELEvalError`` if it doesn't work.

### Example Method

```
    func = self.functions["_?_:_"]
    cond_value, left, right = cast(Tuple[Result, Result, Result], self.visit_children(tree))
    try:
        return func(cond_value, left, right)
    except TypeError as ex:
        logger.debug(f"{func.__name__}({left}, {right}) --> {ex}")
        err = (
            f"found no matching overload for _?_:_ "
            f"applied to '({type(cond_value)}, {type(left)}, {type(right)})'"
        )
        value = CELEvalError(err, ex.__class__, ex.args, tree=tree)
        value.__cause__ = ex
        return value
```

## Python Techniques

- Handling ``CELEvalError``

- Go-isms

- The ``celtypes`` module

### Handling ``CELEvalError``

CEL has commutative short-circuit operators.

  - (Unlike Python which is left-to-right.)

``(2 / 0 > 3 ? false : true) || true`` is ``true``.

-  The ``2 / 0`` is an division by ezro evaluation error 

-  The error propagates through ``(2 / 0 > 3 ? false : true)``

Most operators propagate a ``CELEvalError``.

The ``||``, ``&&``, and ``?:`` implementations ignores the ``CELEvalError``.

### Go-isms

Python ``//`` truncates toward negative infinity. 

Go ``/`` truncates toward zero.

This also changes how ``%`` modulo works. 

This requires fiddling with signs a bit to get Go answers out of Python.

### celtypes type definitions

We provide all of these:

-   BoolType
-   BytesType
-   DoubleType
-   DurationType
-   IntType
-   ListType
-   MapType
-   NullType
-   StringType
-   TimestampType
-   TypeType
-   UintType

### celtypes details

Here's part of the definition of ``celtypes.DoubleType``.

```
class DoubleType(float):
    def __truediv__(self, other: Any) -> 'DoubleType':
        if cast(float, other) == 0.0:
            return DoubleType("inf")
        else:
            return DoubleType(super().__truediv__(other))
```

It's essentially ``float``

- But... ``a / 0`` in CEL becomes infinity instead of an exception. 

## Extension Functionsimport celpy
env = celpy.Environment()
ast = env.compile("355. / 113.")
prgm = env.program(ast)
prgm.evaluate({})    >>> cel_source = """
    ... i.greet(you)
    ... """

CEL Compile binds functions to the expression.  

Built-in functions for each operator.

Plus. Library of add-on functions available as part of compile

Our CEL source, buried in a filter clause.

In [7]:
cel_source = """
i.greet(you)
"""

This has two globals, `i` and `you`. It expects an external function, `greet`.
We need to introduce these names into the environment. 

(This follows the design pattern for Go. It's not *technically* required for Python.)

For C7N integration, these globals are `Resource`, `Event`, and `Now`. The `CELFilter` class has these wired in and will provide them as part of policy execution.

In [8]:
import celpy
decls = {
    "i": celpy.celtypes.StringType,
    "you": celpy.celtypes.StringType,
    "greet": celpy.celtypes.FunctionType}
env = celpy.Environment(annotations=decls)

Given these names, we can compile our original CEL source to create an AST.  The `CELFilter` class genereally does this to validate the CEL code. 

In principle, the additional names and types are required for function type checking. We do not do this in Python in the current release.

In [11]:
ast = env.compile(cel_source)

Here's an extension function `greet()`. For C7N integration, the extension functions are part of `c7nlib`.  There are a lot of functions available.

In [12]:
def greet(lhs: celpy.celtypes.StringType, rhs: celpy.celtypes.StringType) -> celpy.celtypes.StringType:
    return "Hello {1:s}! Nice to meet you, I'm {0:s}.\\n".format(lhs, rhs)

We can  build the executable from the AST and the extension functions. The `CELFilter` class does this and caches it.

In [13]:
prgm = env.program(ast, functions=[greet])

When evaluating the filter, the `CELFilter` class builds an activation with the global values in it. These are generally the `Resource`, the `Event`, and `Now`. 

In [14]:
activation = {
    "i": "CEL", 
    "you": "world"
}

In [15]:
prgm.evaluate(activation)

"Hello world! Nice to meet you, I'm CEL.\\n"

The `i.greet(you)` CEL expression was evaluated with a definition of `i`, `you`, and the `greet(lhs, rhs)` function.

In case you're curious, here's the timing for this.

In [28]:
%%timeit

prgm.evaluate(activation)

503 µs ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


This gives a common start-up cost of about 500µs for each invocation. This involves a function evaluation and the implementation of the `greet()` method

## Summary

C7N Filters can be restated in CEL.

- A variety of operators and data types.

- Logic is familiar-looking -- similar to C++, Go, Java, or bash ``expr``.

- Sophisticated map/filter/existence macros available.

- Reasonably transparent mapping from AWS Describe JSON to CEL Map object.

- Extension functions allow access to C7N Internals as well as more complex Python functions that can't be expressed succinctly in CEL.